In [8]:
from pathlib import Path
from pyexplain.benchmark.file_utils import *
from pyexplain.benchmark.plot import *
from pyexplain.benchmark.check_results import *
import IPython
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 100)

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['font.weight']= 'bold'

# MAC
environment = 'MAC'
BASE_MAC_LINUX = {
    'MAC': '/Users/emiliogamba/Documents/01_VUB/01_Research/01_Shared_Projects',
    'LINUX': '/home/emilio/research/'
}

PATH_FIGURES_POST_PAPER = Path(BASE_MAC_LINUX[environment]) / "01_holygrail/latex/journal/jair21/figures/"
EXPERIMENT_RESULTS = Path(BASE_MAC_LINUX[environment]) / "06_HPC_Experiments/experiments/data/output/"
BASE_OUTPUT_PATH = BASE_MAC_LINUX[environment] + "/06_HPC_Experiments/experiments/data/output/"
REMOTE_EXPERIMENT_RESULTS = "/data/brussel/101/vsc10143/hpc_experiments2/experiments/data/output/"

# Reviewer: Shouldn't incremental solving be worse than non-incremental?

In [9]:
output_folder = EXPERIMENT_RESULTS / "JAIR_REVIEWS_INCREMENTAL_HS/2023062209/"

df_ocus_non_incr = folder_to_pandas_df_pickle([output_folder])

renaming_expl_config = {
    "OCUS_NOT_INCREMENTAL":"OCUS",
    "OCUS_NOT_INCREMENTAL_HS":"OCUS_NON_INCR_HS",
}
df_ocus_non_incr["explanation config"] = df_ocus_non_incr.apply(lambda row: renaming_expl_config[row["params_explanation_computer"]], axis=1)

In [10]:
df_ocus_non_incr.columns

Index(['time_totalTime', 'time_timeout', 'time_timedout', 'time_explain',
       'time_cumul_explain', 'time_preprocess', 'time_opt', 'time_sat',
       'time_grow', 'time_disj_mcs', 'time_remaining', 'numbers_explained',
       'numbers_#opt', 'numbers_#sat', 'numbers_#grow', 'numbers_#disj_mcs',
       'numbers_#hs', 'numbers_#hs_disj_mcs', 'explanation', 'params_output',
       'params_instance', 'params_timeout', 'params_explanation_computer',
       'params_grow', 'params_maxsatpolarity', 'params_interpretation',
       'params_weighing', 'params_reuse_SSes', 'params_sort_literals',
       'params_disable_disjoint_mcses', 'params_disjoint_mcses',
       'params_disjoint_mcs_interpretation', 'params_disjoint_mcs_weighing',
       'time_timedout_initial', 'time_totalTime_initial',
       'explanation config'],
      dtype='object')

# Authors: No, so what's the average speed up over all instances?

Ideally, a fair comparison of the speed-up for keeping the MIP solver warm or not is: 

    "Generate the same explanation step with": 

        - OCUS hs solver restart at every iteration
        - OCUS hs solver warm

Then compare the speed-up that is acquired at every iteration (something we can see in time explain ?).

This would allow us to include timedout instances.

## 1. Cumulative impact of warm MIP solver on the generation of a whole explanation sequence

In [12]:
selected_columns = [
    "params_explanation_computer", 
    "params_instance", 
    "time_totalTime"]


,time_totalTime
params_explanation_computer,
OCUS_NOT_INCREMENTAL,288.145194
OCUS_NOT_INCREMENTAL_HS,639.711834


In [22]:
def time_explain_speed_up(t_expls_not_incr, t_expls_incr):
    speed_ups = [t_expl_not_incr/t_expl_incr for t_expl_not_incr, t_expl_incr in zip(t_expls_not_incr, t_expls_incr)]
    #print(f"#speedups: {len(speed_ups)} -- #not incr: {len(t_expls_not_incr)} ----- # incr {len(t_expls_incr)}")
    return speed_ups

all_speed_ups = []
avg_speed_ups = []

for instance in df_ocus_non_incr["params_instance"].unique():
    df_puzzle = df_ocus_non_incr[
        df_ocus_non_incr["params_instance"] == instance
    ]
    assert len(df_puzzle) == 2, f"SHould be 2 configurations MIP incr and not incr.\n{df_puzzle}"
    
    t_expls_incr = df_puzzle[
        df_puzzle["params_explanation_computer"] == "OCUS_NOT_INCREMENTAL"
    ]["time_explain"].to_list()[0]
    t_expls_not_incr = df_puzzle[
        df_puzzle["params_explanation_computer"] == "OCUS_NOT_INCREMENTAL_HS"
    ]["time_explain"].to_list()[0]
    
    expl_speed_ups = time_explain_speed_up(
        t_expls_not_incr=t_expls_not_incr, 
        t_expls_incr=t_expls_incr)
    if len(expl_speed_ups) == 0:
        continue
    
    avg_speed_ups.append(np.mean(expl_speed_ups))
    all_speed_ups += expl_speed_ups
# print(avg_speed_ups)
print(np.mean(all_speed_ups))
print(np.min(all_speed_ups))
print(np.median(all_speed_ups))
print(np.max(all_speed_ups))


3.6914504470021674
0.01029385281678945
1.7292997568438269
533.3835012149706


## 2. Impact of warm MIP solver on the generation of a whole explanation sequence